In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import torch

import matplotlib.pyplot as plt

from torch.utils.data import Dataset
import torch.nn as nn
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam

from scipy.stats import spearmanr

from sklearn.metrics import mean_squared_error
from sklearn import metrics

import os
from collections import Counter


In [14]:
#load training data (will be put in a function later)  
path = os.getcwd()
for i in range(3) :

    path = os.path.dirname(path)

path += '/data/'
train_df = pd.read_csv(path + 'train_v1.csv',index_col="seq_id")
train_df = train_df.drop(columns=['data_source'])
train_df = train_df.dropna()
train_df.head()

,protein_sequence,pH,tm
seq_id,,,
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5


In [15]:
def split_data(df,tm,ratio):
    num_row = len(df)
    indices = np.random.permutation(num_row)
    index_split = int(np.floor(ratio * num_row))
    index_tr = indices[:index_split]
    index_te = indices[index_split:]

    x_tr = df.iloc[index_tr]
    x_te = df.iloc[index_te]
    y_tr = tm.iloc[index_tr]
    y_te = tm.iloc[index_te]
    
    return y_tr, x_tr, y_te, x_te


In [16]:
train_df['length'] = train_df['protein_sequence'].str.len()
train_df['length'].value_counts()
train_df['length'].nunique()
#print(train_df)
train_df_without_tm = train_df.copy()
del train_df_without_tm['tm']
print(train_df_without_tm)

                                         protein_sequence   pH  length
seq_id                                                                
0       AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...  7.0     341
1       AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...  7.0     286
2       AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...  7.0     497
3       AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...  7.0     265
4       AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...  7.0    1451
...                                                   ...  ...     ...
31385   YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...  7.0     549
31386   YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...  7.0     469
31387   YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...  7.0     128
31388   YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...  7.0     593
31389   YYVPDEYWQSLEVAHKLTFGYGYLTWEWVQGIRSYVYPLLIAGLYK...  7.0     537

[28695 rows x 3 columns]


In [17]:
def Scoring(df_te, df_predicted):
    df = {
    "true": df_te['tm'],
    "predicted": df_predicted['tm']
}
    pearson = df.corr(method='pearson')
    rmse = mean_squared_error(df_te['tm'], df_predicted['tm'], squared=False)
    auc = metrics.roc_auc_score(df_te['tm'], df_predicted['tm'])
    
    print('Pearson: %.3f, RMSE %.3f, AUC: %.3f' %(pearson, rmse, auc))
    return pearson, rmse, auc

In [18]:
y_tr, x_tr, y_te, x_te = split_data(train_df_without_tm, train_df['tm'],0.8)

In [19]:
# create an index of different proteins 
# we need to focus our model training to learn how mutations affect melting point
'''import difflib
ratio = difflib.SequenceMatcher(None, train_df['protein_sequence'][i], train_df['protein_sequence'][j]).ratio()
'''





"import difflib\nratio = difflib.SequenceMatcher(None, train_df['protein_sequence'][i], train_df['protein_sequence'][j]).ratio()\n"

In [20]:

#if torch.cuda.is_available():  
#print('RAPIDS version',cuda.__version__)

print('Train shape:', train_df.shape )
train_df.head()

Train shape: (28695, 4)


,protein_sequence,pH,tm,length
seq_id,,,,
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7,341
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5,286
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5,497
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2,265
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5,1451


In [21]:
vc = train_df.length.value_counts()
vc.head()

164    748
231    318
455    245
155    243
148    241
Name: length, dtype: int64

In [22]:
#find minimum number operations to convert str1 to str2 (FROM GEEKSFORGEEKS)

def edit_distance(str1, str2):
	
	len1 = len(str1)
	len2 = len(str2)

	# Create a DP array to memoize result
	# of previous computations
	DP = [[0 for i in range(len1 + 1)]
			for j in range(2)];

	# Base condition when second String
	# is empty then we remove all characters
	for i in range(0, len1 + 1):
		DP[0][i] = i

	# Start filling the DP
	# This loop run for every
	# character in second String
	for i in range(1, len2 + 1):
		
		# This loop compares the char from
		# second String with first String
		# characters
		for j in range(0, len1 + 1):

			# If first String is empty then
			# we have to perform add character
			# operation to get second String
			if (j == 0):
				DP[i % 2][j] = i

			# If character from both String
			# is same then we do not perform any
			# operation . here i % 2 is for bound
			# the row number.
			elif(str1[j - 1] == str2[i-1]):
				DP[i % 2][j] = DP[(i - 1) % 2][j - 1]
			
			# If character from both String is
			# not same then we take the minimum
			# from three specified operation
			else:
				DP[i % 2][j] = (1 + min(DP[(i - 1) % 2][j],
									min(DP[i % 2][j - 1],
								DP[(i - 1) % 2][j - 1])))
			
	# After complete fill the DP array
	# if the len2 is even then we end
	# up in the 0th row else we end up
	# in the 1th row so we take len2 % 2
	# to get row
	print(DP[len2 % 2][len1], "")



str1 = train_df['protein_sequence'][0]
str2 = train_df['protein_sequence'][1]		

edit_distance(str1, str2)


267 


In [25]:
def edit_distance_matrixx(df):
    for i in range(len(df)):
        for j in range(len(df)):
            df[i][j] = edit_distance(df[i], df[j])
    return df

In [33]:
train_df['group'] = -1
grp = 0


# MUTATION THRESHOLD
M_THRESHOLD = 10
# INSERTION DELETION THRESHOLD
D_THRESHOLD = 3

for k in range(len(vc)):
    c = vc.index[k]
    # SUBSET OF TRAIN DATA WITH SAME PROTEIN LENGTH PLUS MINUS D_THRESHOLD
    tmp = train_df.loc[(train_df.length>=c-D_THRESHOLD)&(train_df.length<=c+D_THRESHOLD)&(train_df.group==-1)]
    if len(tmp)<=1: break
    # COMPUTE LEVENSTEIN DISTANCE
    print(tmp.protein_sequence)
    #x = tmp.protein_sequence.str.edit_distance_matrix()
    x = edit_distance_matrixx(tmp.protein_sequence)
    x = np.array( x.to_pandas().values.tolist() )
    # COUNT HOW MANY MUTATIONS WE SEE
    mutation = []
    for kk in range(1,M_THRESHOLD+1):
        mutation.append( len( np.unique( np.where( x==kk )[0] ) ) )
    # FIND RELATED ROWS IN TRAIN WITH M_THRESHOLD MUTATIONS OR LESS
    y = np.unique( np.where( (x>0)&(x<=M_THRESHOLD) )[0] )
    seen = []
    for j in y:
        if j in seen: continue
        i = np.where( np.array(x[j,])<=M_THRESHOLD )[0]
        seen += list(i)
        idx = tmp.iloc[i].index
        train_df.loc[idx,'group'] = grp
        grp += 1
    ct = vc.iloc[k]
    ct2 = len(tmp)
    print(f'k={k} len={c} ct={ct} ct2={ct2} dist_ct={mutation}')
    #if k==9: break

seq_id
9        AAFQVTSNEIKTGEQLTTSHVFSGFGCEGGNTSPSLTWSGVPEGTK...
194      AGDGARQVWEVRARPGQRFSARRSASMFAVGCSMGPFLHFWYLWLD...
274      AKNFEDVRCKCICPPYKENPGHIYNKNISQKDCDCLHVVEPMPVRG...
381      APCPINGLPIVRNISDLPQDNYGRPGLSHMTVAGSVLHGMKEVEIW...
386      APFSFYNVVSSDPPLLSISVNRTEGRQKDTARNAVENGEFVVHVSD...
                               ...                        
31023    VLNEHNQKLEWIVGKWRSEFSGKIFWPTVPTMTFGEELVINEAPIA...
31113    VSQHLIHPLMGVSLALQSHVRELAALLRMKDLEIQAYQESGAVLSR...
31147    VVIPSPLKSSYQSIAGQRAINLDVNVRLLTVLALQQSGWQLYEDHW...
31154    VVPLEARRAFHMTKDMLPGSYPRTPEERAAAAKKYNMRVEDYEPYP...
31367    YTVSSFQKSLDLAKPCKRFVLHLHDIAYDGDNAANATSAAIVNPLG...
Name: protein_sequence, Length: 1032, dtype: object


KeyError: 0

In [31]:
# display code
for k in range(10):
    print('#'*25)
    print(f'### GROUP {k}')
    print('#'*25)
    display( train_df.loc[train_df.group==k] )

#########################
### GROUP 0
#########################


,protein_sequence,pH,tm,length,group
seq_id,,,,,


#########################
### GROUP 1
#########################


,protein_sequence,pH,tm,length,group
seq_id,,,,,


#########################
### GROUP 2
#########################


...

,protein_sequence,pH,tm,length,group
seq_id,,,,,


#########################
### GROUP 8
#########################


,protein_sequence,pH,tm,length,group
seq_id,,,,,


#########################
### GROUP 9
#########################


,protein_sequence,pH,tm,length,group
seq_id,,,,,


In [32]:
nrow = train_df.loc[train_df.group==-1].shape[0] 
mx = train_df.group.max() 
train_df.loc[train_df.group==-1,'group'] = np.arange(nrow) + mx + 1.